# Automated Transient Determination

In [2]:
from __future__ import division, print_function, unicode_literals
import os
import os.path
import glob
import itertools
import subprocess
import ccdproc
from astropy import units as u
import numpy as np
import astropy.io.fits as fits
import sys
import shutil as shutil
from shutil import move
import pyfits
from astropy.convolution import Gaussian2DKernel, convolve
import ephem

### Here we can add the below process to the existing Transients code

In [2]:
#This section tells the code where the files are. 
#data_path should point to the directory that holds the .resamp.fits files 
#produced by swarp (usually by Jack's Code)
data_path = ('./output/Temp/')

#master_file is the final processed image made from the above resamp files.
master_file = ['./output/coadd_ngc300_r.fits']

#master_weight is the weight file which corresponds to the above master file.
master_weight = ['./output/coadd_ngc300_r.weight.fits']

In [3]:
#These variables are lists of the resampled files and their corresponding weights.

resamps=sorted(glob.glob(data_path+'*.resamp.fits'))
weights=sorted(glob.glob(data_path+'*.resamp.weight.fits'))

In [10]:
#Creates a segmentation map from the master file. This will be used to generate flag files later.
#To tweak the sensitivity of this segmentation map (brightness of objects it identifies for flagging), change
#the DETECT_THRESH and ANALYSIS_THRESH values in segmentation.sex, located in the config_files directory.
#This file is a temporary file, but the code will not delete it by default, as it can be a useful analysis tool.

subprocess.call('sex '+master_file[0]+' -c ./config_files/segmentation.sex -WEIGHT_IMAGE '+master_weight[0], shell=True)
subprocess.call('mv ./Processed_Images/Temp/check.fits ./Processed_Images/Temp/segmentation_map_ref.fits', shell=True)
master_segmentation = ['./Processed_Images/Temp/segmentation_map_ref.fits']

In [12]:
#This loop creates all the difference images and their respective object, background, and aperture maps.

for i in range(0,2):#len(resamps)):
    reference=fits.open(master_file[0])
    resampled=fits.open(resamps[i])
    
    #measures the distance from the central WCS pixel to the right and to the bottom of the image, respectively.
    resdistx=(resampled[0].header['NAXIS1']-resampled[0].header['CRPIX1'])
    resdisty=(resampled[0].header['NAXIS2']-resampled[0].header['CRPIX2'])
    
    #Creates difference values that represent the start and end pixels of the resampled image on the reference image.
    ref_x_start=int(reference[0].header['CRPIX1']-resampled[0].header['CRPIX1'])
    ref_y_start=int(reference[0].header['CRPIX2']-resampled[0].header['CRPIX2'])
    ref_x_end=int(reference[0].header['CRPIX1']+resdistx)
    ref_y_end=int(reference[0].header['CRPIX2']+resdisty)
    
    #loading files into memory
    refimg=pyfits.open(master_file[0])
    resimg=pyfits.open(resamps[i])
    segimg=pyfits.open(master_segmentation[0])
    
    #converting to array
    D1=refimg[0].data
    D2=resimg[0].data
    D3=segimg[0].data
    
    #resizing and scaling pixel values
    resize_ref=D1[ref_y_start:ref_y_end,ref_x_start:ref_x_end]
    resamp_scaled=D2*resampled[0].header['FLXSCALE']
    resize_seg=D3[ref_y_start:ref_y_end,ref_x_start:ref_x_end]
    
    #doing the subtraction of the reference image from the resampled
    out_file=resamp_scaled-resize_ref
    
    #loading header information from original resampled image. This will be saved into the new images.
    head=pyfits.getheader(resamps[i])
    
    #loads the filename of the resamp image into a variable, then deletes .resamp.fits from the end of it.
    #This allows the code to write new files with the same naming format, but different file extension names.
    #If your files don't end in .resamp.fits, the -12 will need to be changed to match the number of characters
    #in the file extension.
    path=os.path.basename(resamps[i])
    new_path=path[:-12]
    
    #saves the final difference file, and the temporary resized segmentation map.
    pyfits.writeto(('./Processed_Images/'+new_path+'.difference.fits'), out_file, head)
    pyfits.writeto(('./Processed_Images/Temp/temp_segment.fits'), resize_seg, head)
    
    #produces a flag map from the temporary segmentation map.
    subprocess.call('ww -c ./config_files/default.ww -WEIGHT_NAMES '+weights[i]+',./Processed_Images/Temp/temp_segment.fits', shell=True)
    
    #does the main sextractor run on the subtracted file, producing the object, background, and aperture files.
    #this is controlled by default.sex in the config_files directory
    subprocess.call('sex ./Processed_Images/'+new_path+'.difference.fits -c ./config_files/default.sex', shell=True)
    
    #cleans up temporary files and renames background, object, and aperture files. 
    subprocess.call('rm ./Processed_Images/Temp/temp_segment.fits',shell=True)
    subprocess.call('mv ./Processed_Images/check.fits ./Processed_Images/'+new_path+'.background.fits',shell=True)
    subprocess.call('mv ./Processed_Images/check2.fits ./Processed_Images/'+new_path+'.object.fits',shell=True)
    subprocess.call('mv ./Processed_Images/check3.fits ./Processed_Images/'+new_path+'.apertures.fits',shell=True)
    subprocess.call('rm ./Processed_Images/Temp/weight.fits',shell=True)
    subprocess.call('rm ./Processed_Images/Temp/flag.fits',shell=True)
#Uncomment the next line if you would like the code to automatically clean up the master segmentation map.
#subprocess.call('rm ./Processed_Images/Temp/segmentation_map_ref.fits',shell=True)

0

# Begin Brendan's Transient checking script:

In [ ]:
# this is where the code can be tacked onto the pipeline, after subtracting reference images.
# At this point we should have a combined image, with reference subtracted, where the only point sources remaining
# (in theory) are transients such as supernovae or asteroids.

# This code aims to classify such sources as either known asteroids or anomalies, which can then be manually checked.

### Run SExtractor once more to spot remaining (transient) sources:

Generate list of sources:

## Check against catalogues of known asteroids:

In [2]:
# Download latest MPCORB catalogue from:
# www.minorplanetcenter.org/iau/MPCORB/MPCORB.DAT

# Full list of paramters and how to read the data:
# http://www.minorplanetcenter.net/iau/info/MPOrbitFormat.html


# and do a manual check using PYEPHEM.

### Fiddling around with pyephem:

In [133]:
# The following 3 date entries should be indentical, but the preferred method (#3 below) is not recognised 
# for some reason...:

# date method 1:
print(ephem.Date(35497.7197916667))


# date method 2:
# Note: careful of  double parentheses; for some reason they are required.
print(ephem.Date((1997, 11, 13, 5, 16, 30.0)))


# date method 3:
# this SHOULD return the same value as the above two.. not sure. will have to work around for now.
print(ephem.Date('1997/3/10 05:16:30'))

1997/3/10 05:16:30
1997/11/13 05:16:30


ValueError: dates must be initialized from a number, string, tuple, or datetime

In [43]:
# Checking ephem.separation() function at time of Mercury transit.

m = ephem.Mercury()
m.compute((2006, 11, 8, 21, 41, 0))

n = ephem.Sun()
n.compute((2006, 11, 8, 21, 41, 0))

print('%f %f' % (m.ra, m.dec))
print ('%f %f' % (n.ra, n.dec))
sep = ephem.separation(m,n)
print(sep)
print(ephem.degrees(sep))
# Angular separation < 30 arcmin = transit :)
# she works!

3.906462 -0.293787
3.907369 -0.291929
0:07:03.0
0:07:03.0


In [54]:
if sep < ephem.degrees(0.00581776):
    print('the threshold is ' + str(ephem.degrees(0.00581776)))
    print('separation smaller than threshold.')

the threshold is 0:20:00.0
separation smaller than threshold.


In [207]:
m = ephem.Mars()
d1 = ephem.Date((2009, 2, 2, 0, 0, 0))
m.compute(d1)
print(('%s  %s') %(m.ra, m.dec))

20:00:56.48  -21:30:08.1


### Import MPCORB database and split into a table:

In [132]:
# Load MPCORB.DAT data into an array:
# delimiter to split data into required columns
# Add 'skip_header=41' parameter when using real data to skip header info
# length of line = 202 characters
# 
# names = ['designation', 'abs mag', 'slope param', 'epoch', 'mean epoch anomaly', 'perihelion arg', 'longitude', /
# 'inclination', 'eccentricity', 'mean daily motion', 'semimajor axis', 'uncertainty', 'reference', 'num obs', /
# 'num opp', 'obs years / arc length', 'rms', 'coarse perturb', 'precise perturb', 'comp name', '4hex flag', / 
# 'readable designation', 'last obs']

MPCORB = np.genfromtxt('MPCORB/MPCORB_test', autostrip=True, dtype=str,  delimiter = [8,6,6,6,10,11,11,11,11,12,12,3,10,6,4,10,5,4,4,11,5,28,8])

designation = MPCORB[:,0]
abs_mag = MPCORB[:,1]
slope_param = MPCORB[:,2]
epoch = MPCORB[:,3]
mean_epoch_anomaly = MPCORB[:,4]
perihelion_arg = MPCORB[:,5]
longitude = MPCORB[:,6]
inclination = MPCORB[:,7]
eccentricity = MPCORB[:,8]
mean_daily_motion = MPCORB[:,9]
semimajor_axis = MPCORB[:,10]
uncertainty = MPCORB[:,11]
reference = MPCORB[:,12]
num_obs = MPCORB[:,13]
num_opp = MPCORB[:,14]
obs_years_arc_length = MPCORB[:,15]
rms = MPCORB[:,16]
coarse_perturb = MPCORB[:,17]
precise_perturb = MPCORB[:,18]
comp_name = MPCORB[:,19]
hex_flag =  MPCORB[:,20]
read_des = MPCORB[:,21]
last_obs = MPCORB[:,22]

### Define a function to extract the packed Epoch date format given in the MPCORB data:

In [119]:
# define dictionary of alpha to numbers for MPCORB packed dates translation:
# see  <http://www.minorplanetcenter.net/iau/info/PackedDates.html>  for more info.

def epoch_convert(date):
    
    "Converts the MPCORB Epoch from packed form to a regular YYYYMMDD.DDDD string to be split up and used later."
    
    packeddates = {'1':'1', '2':'2', '3':'3', '4':'4', '5':'5', '6':'6', '7':'7', '8':'8', '9':'9', \
                   '0':'0', 'A':'10', 'B':'11', 'C':'12', 'D':'13', 'E':'14', 'F':'15', 'G':'16', \
                   'H':'17', 'I':'18', 'J':'19', 'K':'20', 'L':'21', 'M':'22', 'N':'23', 'O':'24', \
                   'P':'25', 'Q':'26', 'R':'27', 'S':'28','T':'29', 'U':'30', 'V':'31' \
              }
    datestring = list(date[0:5])
    datestring2 = ""
    nums = "0123456789"
    
    # Conditionals to distinguish between packed dates, eg  avoid confusion between 1-Nov (11 1) /
    # and 11-Jan (1 11) when compiled back into a string (111). 
    # We convert all single-digit numeric month/day values to 2-digits (eg 6 --> 06):
    
    if date[3] in nums and date[4] in nums:
        datestring.insert(3,'0')
        datestring.insert(5,'0')
        for i in range(len(datestring)):
            char = packeddates['%s' % datestring[i]]
            datestring2 += str(char)
        if len(date) > 5:
            datestring2 += "." + date[5:]
        return datestring2 
    elif date[3] in nums and date[4] not in nums:
        datestring.insert(3,'0')
        for i in range(len(datestring)):
            char = packeddates['%s' % datestring[i]]
            datestring2 += str(char)
        if len(date) > 5:
            datestring2 += "." + date[5:]
        return datestring2
    elif date[3] not in nums and date[4] in nums:
        datestring.insert(4,'0')
        for i in range(len(datestring)):
            char = packeddates['%s' % datestring[i]]
            datestring2 += str(char)
        if len(date) > 5:
            datestring2 += "." + date[5:]
        return datestring2
    elif date[3] not in nums and date[4] not in nums:
        for i in range(len(datestring)):
            char = packeddates['%s' % datestring[i]]
            datestring2 += str(char)
        if len(date) > 5:
            datestring2 += "." + date[5:]
        return datestring2
    
    print(datestring2)

In [124]:
test1 = 'K165B5566'
epoch_convert(test1)

u'20160511.5566'

In [134]:
for packed in epoch:
    print('epoch: ' + epoch_convert(packed))

epoch: 20160113
epoch: 20160113
epoch: 20160113
epoch: 20160113
epoch: 20160113
epoch: 20160113
epoch: 20160113
epoch: 20100723


# NEED TO NOW ASSIGN THE ORBITAL PARAMS TO AN OBJECT.
# CHECK THE RA DEC AT SOME ARBITRARY TIME, SEE IF SHE WORKS.

### Load some asteroid parameters into pyephem to test:

Required orbital params:

N (= Om) = longitude of the ascending node 

i (= inc) = inclination to the ecliptic (plane of the Earth's orbit)

w (= om) = argument of perihelion

a = semi-major axis, or mean distance from Sun

e = eccentricity (0=circle, 0-1=ellipse, 1=parabola)

M = mean anomaly (0 at perihelion; increases uniformly with time)

In [242]:
# initialise a new, blank orbital body and load orbital params:
params = MPCORB[0]
body = ephem.EllipticalBody()

#six required params for Keplerian orbit:
# Longitude of ascending node:
body._Om = float(params[6])
# Inclination:
body._inc = float(params[7])
# Arg of perihelion
body._om = float(params[5])
# Mean distance from Sun: ???????????? ACCURATE?? Semi Major Axis, technically.
body._a = float(params[10])
# Eccentricity:
body._e = float(params[8])
# Mean anomoly from perihelion:
body._M = float(params[4])
# Epoch for _M:
body._epoch_M = ephem.Date((2016, 1, 13, 0, 0, 0))

print(epoch_convert('K161D'))
print(body._epoch_M)

20160113
2016/1/13 00:00:00


In [237]:
#Vesta correct RA DEC for Jan 19, 2016: RA: 0h51m20s, DEC: -1 31' 10"
date1 = ephem.Date((2016, 1, 17, 0, 0, 0))
body.compute(date1)
print(date1)
print('RA: ' +str(body.ra) +', DEC: '+ str(body.dec))

2016/1/17 00:00:00
RA: 0:54:35.42, DEC: -1:19:27.4


In [149]:
# Possible requirement for rectifying discrepencies?
# http://www.stjarnhimlen.se/comp/ppcomp.html#16

# Correction for precession:?
# N = N_Epoch + 0.013967 * ( 2000.0 - Epoch ) + 3.82394E-5 * d

#and mean daily motion n required too:?
#mdm = aasdasdsads


#print(body._Om, body._inc, body._om, body._a, body._e, body._M)?
    

In [ ]:
# test reading fits header for pulling date and time of image and store as string.

In [65]:
light = '2014-09-21_83F011167_12_light.bdfw.fits'
datestr = fits.getheader(light)['DATE']

In [66]:
datestr

'2014-09-21T15:05:59'

In [75]:
print(ephem.Date((int(datestr[0:4]), int(datestr[5:7]), int(datestr[8:10]), int(datestr[11:13]), \
                  int(datestr[14:16]), int(datestr[17:19]))))

2014/9/21 15:05:59


### Search MPCORB for matches:

In [ ]:
def 

In [ ]:
# Nest this loop outside the MPCORB search loop:
# for trans in translist:
    # RA = trans.header["RA"]
    # DEC = trans.header["DEC"]
    # obstime = trans.header["DATE"]
    # identcount = 0
    # for object in MPCORB:
        # .......
    
    # if identcount = 0 (no object found):
        # anom.append("object designation", "object name" "trans WCS at that time", )

In [54]:
ayear = int(datestr[0:4])
amonth = int(datestr[5:7])
aday = int(datestr[8:10])
ahour = int(datestr[11:13])
amin = int(datestr[14:16])
asec = int(datestr[17:19])

In [56]:
print(ephem.Date((ayear, amonth, aday, ahour, amin, asec)))

2014/9/21 15:05:59


In [ ]:
For MP in MPCORB:
    inc = 
     = 
    newobject = []
    
    
    
    # convert orbital params at given date to RA/DEC
    # Vec = vectorize (trans RA & DEC) with (MPCORB object RA & DEC)
    # if Vec <= threshold (20 arcmin?):
        # ident.append("object designation", "object name" "object WCS at that time", "distance vector from trans")
        # identcount += 1
   

### Return anomalies:

In [ ]:
# Use pyephem, take WCS of anomalies, if pyephem of each oject in MPCORB is within a threshold (vectorize) at that time, then return object
# else, return anomaly

In [52]:
print(len('00001    3.34  0.12 K161D 181.38133   72.73324   80.32180   10.59166  0.0757544  0.21400734   2.7681117  0 MPO358623  6592 109 1801-2015 0.60 M-v 30h MPCLINUX   0000      (1) Ceres              20151128'))

202
